## DEMO

### 01 load model

In [1]:
import sys
sys.path.append("../../../deepyeast/")
import os

import numpy as np
from tqdm import trange
from skimage.io import imsave

from utils import load_model
from postprocess.post_process_utils import post_process_panoptic

In [2]:
model = load_model("../../../model_dir/deepyeast_001/", "../../configs/config_wl.yaml")

2023-01-03 02:06:55.686062: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 02:07:00.185924: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-03 02:07:11.018351: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /dcsrsoft/spack/arolle/v1.0/spack/opt/spack/linux-rhel8-zen/gcc-8.4.1/gcc-10.4.0-7fjdev7tbifqfonsjwlqmr6qcnza2ezu/lib64:/dcsrsoft/spack/arolle/v1.0/spack/opt/spack/linux-rhel8-zen/gcc-8.4.1/gcc-10.4.0-7fjdev7tbif

restoring or initializing model...
restored model from ../../../model_dir/deepyeast_001/ckpt-63000.
restored from checkpoint: ../../../model_dir/deepyeast_001/ckpt-63000


### 02 load image and prediction mask

In [3]:
from nd2reader import ND2Reader
from analysis.utils import file_traverse

In [4]:
file_list = file_traverse("../../../../data/AV_SEG/Exponential-BF01/")
save_path = './test' 

In [8]:
for i in range(0, len(file_list)):
    file = file_list[i]
    basename = os.path.basename(file)[:-4]
    
    images=ND2Reader(file)
    img = np.array(images)
    predict = np.zeros(img.shape)
    print(basename)

    for f in trange(img.shape[0]):
        frame = img[f]
        output = model.predict(frame)
        mask = post_process_panoptic(output["panoptic_pred"][0].numpy(), area_threshold=300)
        predict[f] = mask
    imsave(os.path.join(save_path, basename+"_mask.tif"),predict)

012_EX01


100%|██████████| 11/11 [02:10<00:00, 11.89s/it]
